In [ ]:
# Imports
import requests
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from bs4 import BeautifulSoup
import plotly.io as pio
from sklearn.covariance import LedoitWolf
import re

***RIESGO PAIS ARGENTINA***

In [ ]:
# Riesgo País
prima_riesgo_mas_reciente = None
fecha_actualizacion = None
fuente_datos = None

hoy = datetime.now()
hace_dos_anios = hoy - timedelta(days=730)
fecha_inicio = hace_dos_anios.strftime('%d-%m-%Y')
fecha_fin = hoy.strftime('%d-%m-%Y')

url_ambito = f"https://mercados.ambito.com/riesgopais/historico-general/{fecha_inicio}/{fecha_fin}"

headers_completos = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'es-ES,es;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://mercados.ambito.com/',
    'Connection': 'keep-alive',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin'
}

try:
    session = requests.Session()
    response = session.get(url_ambito, headers=headers_completos, timeout=15)
    
    if response.status_code == 200:
        datos = response.json()
        df = pd.DataFrame(datos[1:], columns=datos[0])
        df['Puntos'] = df['Puntos'].astype(str).str.replace(',', '.')
        df['Puntos'] = pd.to_numeric(df['Puntos'])
        df['Fecha'] = pd.to_datetime(df['Fecha'], format='%d-%m-%Y')
        df = df.sort_values('Fecha', ascending=False).reset_index(drop=True)
        
        prima_riesgo_mas_reciente = df.iloc[0]['Puntos'] / 100
        fecha_actualizacion = df.iloc[0]['Fecha'].strftime('%d/%m/%Y')
        fuente_datos = "Ámbito Financiero"
    else:
        print(f"⚠️ Ámbito respondió con código {response.status_code}. Intentando Rava...")
except Exception as e:
    print(f"⚠️ Error en Ámbito: {str(e)}. Intentando Rava...")

if prima_riesgo_mas_reciente is None:
    try:
        print("🔄 Intentando Rava...")
        url_rava = "https://www.rava.com/perfil/riesgo%20pais"
        
        headers_rava = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
            'Accept-Language': 'es-AR,es;q=0.9,en;q=0.8',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1'
        }
        
        session_rava = requests.Session()
        response_rava = session_rava.get(url_rava, headers=headers_rava, timeout=15, allow_redirects=True)
        
        if response_rava.status_code == 200:
            soup = BeautifulSoup(response_rava.content, 'html.parser')
            all_text = soup.get_text()
            valor_encontrado = None
            
            price_elem = soup.find('div', class_='price')
            if price_elem:
                texto_price = price_elem.get_text(strip=True)
                numeros_solo = re.sub(r'[^\d,\.]', '', texto_price)
                match = re.search(r'(\d{3,4})[,\.]?(\d{2})?', numeros_solo)
                if match:
                    valor_encontrado = match.group(1) + '.' + (match.group(2) if match.group(2) else '00')
            
            if not valor_encontrado:
                matches = re.findall(r'(\d{3,4})[,\.](\d{2})', all_text)
                if matches:
                    for match in matches:
                        valor_num = float(match[0] + '.' + match[1])
                        if 300 <= valor_num <= 3000:
                            valor_encontrado = match[0] + '.' + match[1]
                            break
            
            if valor_encontrado:
                prima_riesgo_mas_reciente = float(valor_encontrado) / 100
                fecha_actualizacion = datetime.now().strftime('%d/%m/%Y')
                fuente_datos = "Rava Bursátil"
        else:
            print(f"⚠️ Rava respondió con código {response_rava.status_code}")
    except Exception as e:
        print(f"❌ Error en Rava: {str(e)}")

if prima_riesgo_mas_reciente is None:
    print("\n" + "="*70)
    print("❌ ERROR: No se pudo obtener el Riesgo País automáticamente.")
    print("="*70)
    print("\n💡 SOLUCIÓN: Ingresa el valor manualmente desde alguna de estas fuentes:")
    print("   📍 https://www.ambito.com/contenidos/riesgo-pais.html")
    print("   📍 https://www.rava.com/perfil/riesgo%20pais")
    print("\n")
    
    try:
        valor_manual = input("Ingresa el Riesgo País en puntos básicos (ej: 650): ")
        if valor_manual and valor_manual.strip():
            valor_limpio = re.sub(r'[^\d\.]', '', valor_manual)
            if valor_limpio:
                valor_puntos = float(valor_limpio)
                # Convertir puntos básicos a porcentaje: 650 puntos = 6.5%
                prima_riesgo_mas_reciente = valor_puntos / 100
                
                fecha_actualizacion = datetime.now().strftime('%d/%m/%Y')
                fuente_datos = "Ingreso Manual"
    except Exception as e:
        print(f"Error al procesar input: {str(e)}")

if prima_riesgo_mas_reciente is None:
    raise RuntimeError("❌ ERROR CRÍTICO: No se pudo obtener el Riesgo País. Por favor, ejecuta la celda nuevamente e ingresa el valor manualmente.")

if prima_riesgo_mas_reciente is not None:
    html = f"""
    <div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        border-radius: 20px;
        padding: 40px;
        box-shadow: 0 10px 30px rgba(0,0,0,0.3);
        text-align: center;
        max-width: 500px;
        margin: 30px auto;
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    ">
        <h2 style="
            color: white;
            font-size: 24px;
            margin-bottom: 10px;
            font-weight: 300;
            letter-spacing: 1px;
        ">Prima de Riesgo País</h2>

        <div style="
            background: rgba(255,255,255,0.95);
            border-radius: 15px;
            padding: 30px;
            margin: 20px 0;
        ">
            <div style="
                font-size: 64px;
                font-weight: bold;
                color: #667eea;
                margin-bottom: 10px;
            ">{prima_riesgo_mas_reciente:.2f}%</div>

            <div style="
                font-size: 14px;
                color: #666;
                margin-top: 15px;
            ">Actualizado: {fecha_actualizacion}</div>
        </div>

        <div style="
            color: rgba(255,255,255,0.8);
            font-size: 12px;
            margin-top: 15px;
        ">Fuente: {fuente_datos}</div>
    </div>
    """

    display(HTML(html))
else:
    print("❌ Error crítico: No se pudo obtener el riesgo país de ninguna fuente.")

***EMPRESAS***

In [ ]:
# Empresas
activos_portfolio = []

banderas_paises = {
    'United States': '🇺🇸', 'Argentina': '🇦🇷', 'Brazil': '🇧🇷', 'Mexico': '🇲🇽',
    'Chile': '🇨🇱', 'Colombia': '🇨🇴', 'Peru': '🇵🇪', 'United Kingdom': '🇬🇧',
    'Germany': '🇩🇪', 'France': '🇫🇷', 'Spain': '🇪🇸', 'Italy': '🇮🇹',
    'Japan': '🇯🇵', 'China': '🇨🇳', 'South Korea': '🇰🇷', 'Canada': '🇨🇦',
    'Australia': '🇦🇺', 'India': '🇮🇳', 'Netherlands': '🇳🇱', 'Switzerland': '🇨🇭',
    'US': '🇺🇸', 'AR': '🇦🇷', 'BR': '🇧🇷', 'MX': '🇲🇽', 'CL': '🇨🇱',
    'CO': '🇨🇴', 'PE': '🇵🇪', 'GB': '🇬🇧', 'DE': '🇩🇪', 'FR': '🇫🇷',
    'ES': '🇪🇸', 'IT': '🇮🇹', 'JP': '🇯🇵', 'CN': '🇨🇳', 'KR': '🇰🇷',
    'CA': '🇨🇦', 'AU': '🇦🇺', 'IN': '🇮🇳', 'NL': '🇳🇱', 'CH': '🇨🇭',
    'Belgium': '🇧🇪', 'Sweden': '🇸🇪', 'Norway': '🇳🇴', 'Denmark': '🇩🇰',
    'Finland': '🇫🇮', 'Ireland': '🇮🇪', 'Portugal': '🇵🇹', 'Austria': '🇦🇹',
    'Israel': '🇮🇱', 'Singapore': '🇸🇬', 'Hong Kong': '🇭🇰', 'Taiwan': '🇹🇼',
    'Thailand': '🇹🇭', 'Malaysia': '🇲🇾', 'Indonesia': '🇮🇩', 'Philippines': '🇵🇭',
    'New Zealand': '🇳🇿', 'South Africa': '🇿🇦', 'Russia': '🇷🇺', 'Turkey': '🇹🇷'
}

output_header = widgets. Output()
output_validacion = widgets.Output()
output_lista = widgets.Output()

input_ticker = widgets.Text(
    placeholder='Ej: AAPL, GGAL, TSLA',
    description='',
    style={'description_width': '0px'},
    layout=widgets.Layout(width='300px', height='45px')
)

btn_agregar = widgets.Button(
    description='Agregar',
    button_style='primary',
    layout=widgets.Layout(width='120px', height='45px'),
    style={'button_color': '#667eea', 'font_weight': 'bold'}
)

btn_finalizar = widgets. Button(
    description='Finalizar Portfolio',
    button_style='success',
    layout=widgets.Layout(width='300px', height='50px', margin='20px 0 0 0'),
    style={'font_weight': 'bold'}
)

def obtener_info_activo(ticker):
    try:
        activo = yf.Ticker(ticker)
        info = activo.info

        nombre = info.get('longName') or info.get('shortName') or ticker
        pais = info.get('country', 'United States')
        emoji_bandera = banderas_paises.get(pais, '🌐')

        return {
            'ticker': ticker. upper(),
            'nombre': nombre,
            'pais': pais,
            'bandera': emoji_bandera,
            'valido': True
        }
    except:
        return {'valido': False}

def actualizar_lista():
    with output_lista:
        clear_output()
        if activos_portfolio:
            html_items = ""
            for i, activo in enumerate(activos_portfolio):
                html_items += f"""
                <div style="
                    background: white;
                    border-left: 4px solid #667eea;
                    padding: 15px 20px;
                    margin: 10px 0;
                    border-radius: 8px;
                    box-shadow: 0 2px 8px rgba(0,0,0,0. 1);
                    display: flex;
                    align-items: center;
                    justify-content: space-between;
                ">
                    <div style="display: flex; align-items: center; gap: 15px;">
                        <span style="font-size: 32px;">{activo['bandera']}</span>
                        <div>
                            <div style="font-weight: bold; font-size: 18px; color: #333;">
                                {activo['ticker']}
                            </div>
                            <div style="color: #666; font-size: 14px;">
                                {activo['nombre']}
                            </div>
                            <div style="color: #999; font-size: 12px; margin-top: 3px;">
                                {activo['pais']}
                            </div>
                        </div>
                    </div>
                    <div style="
                        background: #667eea;
                        color: white;
                        padding: 5px 12px;
                        border-radius: 20px;
                        font-size: 12px;
                        font-weight: bold;
                    ">#{i+1}</div>
                </div>
                """

            display(HTML(f"""
            <div style="max-width: 600px; margin: 20px auto;">
                <h3 style="color: #667eea; text-align: center; margin-bottom: 20px;">
                    📊 Activos en el Portfolio ({len(activos_portfolio)})
                </h3>
                {html_items}
            </div>
            """))

def on_agregar_click(b):
    ticker = input_ticker. value.strip().upper()

    with output_validacion:
        clear_output()

        if not ticker:
            display(HTML("""
            <div style="color: #e74c3c; padding: 10px; text-align: center;">
                ⚠️ Por favor ingresa un ticker
            </div>
            """))
            return

        if any(a['ticker'] == ticker for a in activos_portfolio):
            display(HTML(f"""
            <div style="color: #e74c3c; padding: 10px; text-align: center;">
                ⚠️ {ticker} ya está en el portfolio
            </div>
            """))
            return

        display(HTML("""
        <div style="color: #667eea; padding: 10px; text-align: center;">
            🔍 Validando ticker...
        </div>
        """))

        info = obtener_info_activo(ticker)

        clear_output()

        if info['valido']:
            activos_portfolio.append(info)
            display(HTML(f"""
            <div style="
                background: #d4edda;
                color: #155724;
                padding: 15px;
                border-radius: 8px;
                text-align: center;
                border: 1px solid #c3e6cb;
            ">
                ✅ {info['bandera']} <strong>{info['ticker']}</strong> - {info['nombre']} agregado exitosamente
            </div>
            """))
            input_ticker. value = ''
            actualizar_lista()
        else:
            display(HTML(f"""
            <div style="
                background: #f8d7da;
                color: #721c24;
                padding: 15px;
                border-radius: 8px;
                text-align: center;
                border: 1px solid #f5c6cb;
            ">
                ❌ No se pudo validar el ticker <strong>{ticker}</strong>. Verifica que sea correcto.
            </div>
            """))

def on_finalizar_click(b):
    with output_validacion:
        clear_output()
        if len(activos_portfolio) < 2:
            display(HTML("""
            <div style="
                background: #fff3cd;
                color: #856404;
                padding: 15px;
                border-radius: 8px;
                text-align: center;
                border: 1px solid #ffeaa7;
            ">
                ⚠️ Necesitas al menos 2 activos para crear un portfolio
            </div>
            """))
        else:
            display(HTML(f"""
            <div style="
                background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
                color: white;
                padding: 30px;
                border-radius: 15px;
                text-align: center;
                box-shadow: 0 10px 30px rgba(0,0,0,0.3);
            ">
                <h2 style="margin: 0 0 10px 0;">🎉 Portfolio Configurado</h2>
                <p style="font-size: 18px; margin: 0;">{len(activos_portfolio)} activos listos para análisis</p>
            </div>
            """))

btn_agregar.on_click(on_agregar_click)
btn_finalizar. on_click(on_finalizar_click)

with output_header:
    display(HTML("""
    <div style="
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 30px;
        border-radius: 15px;
        text-align: center;
        margin-bottom: 30px;
        box-shadow: 0 10px 30px rgba(0,0,0,0. 2);
    ">
        <h1 style="color: white; margin: 0 0 10px 0; font-size: 32px;">
            📈 Configuración de Portfolio
        </h1>
        <p style="color: rgba(255,255,255,0. 9); margin: 0; font-size: 16px;">
            Ingresa los tickers de los activos que conformarán tu portfolio
        </p>
    </div>
    """))

input_box = widgets.HBox(
    [input_ticker, btn_agregar],
    layout=widgets.Layout(
        justify_content='center',
        margin='20px 0',
        gap='10px'
    )
)

container = widgets.VBox([
    output_header,
    input_box,
    output_validacion,
    output_lista,
    widgets.HBox([btn_finalizar], layout=widgets.Layout(justify_content='center'))
])

display(container)

***FRONTERA EFICIENTE***

In [ ]:
# Frontera Eficiente
try:
    _ = activos_portfolio
    _ = prima_riesgo_mas_reciente
except NameError:
    print("❌ Error: Las variables 'activos_portfolio' o 'prima_riesgo_mas_reciente' no están definidas.")
    raise NameError("Faltan variables de configuración.")

retornos_esperados_manuales = {}

pio.renderers.default = 'notebook'

def obtener_tasa_libre_riesgo():
    try:
        tnx = yf.Ticker('^TNX')
        datos = tnx.history(period='5d')
        if not datos.empty: return datos['Close'].iloc[-1] / 100
    except: pass

    try:
        fvx = yf.Ticker('^FVX')
        datos = fvx.history(period='5d')
        if not datos.empty: return datos['Close'].iloc[-1] / 100
    except: pass

    try:
        url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/pages/textview.aspx?data=yield"
        r = requests.get(url, timeout=5)
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find('table', {'class': 't-chart'})
        if table:
            rows = table.find_all('tr')
            if len(rows) > 1:
                cols = rows[-1].find_all('td')
                if len(cols) > 10: return float(cols[10].text.strip()) / 100
    except: pass

    return 0.042

def descargar_precios_historicos(tickers, periodo='2y'):
    if not tickers: return pd.DataFrame()
    datos = yf.download(tickers, period=periodo, progress=False, threads=True)

    if isinstance(datos.columns, pd.MultiIndex):
        precios = datos['Adj Close'] if 'Adj Close' in datos else datos['Close']
    else:
        precios = datos['Adj Close'] if 'Adj Close' in datos else datos['Close']

    if isinstance(precios, pd.Series):
        precios = precios.to_frame(name=tickers[0])

    if precios.index.tz is not None:
        precios.index = precios.index.tz_localize(None)

    return precios.dropna()

def obtener_datos_mercado_sp500():
    end = datetime.now()
    start = end - timedelta(days=730)
    sp500 = yf.Ticker('^GSPC').history(start=start, end=end)['Close']
    if sp500.index.tz is not None:
        sp500.index = sp500.index.tz_localize(None)
    return sp500

def calcular_retornos_hibridos(precios, manuales, tasa_libre_riesgo, lista_activos):
    """
    Calcula retornos esperados híbridos (CAPM + Manual) y siempre guarda el CAPM teórico.
    Retorna: (retornos_finales, retornos_capm_teoricos)
    """
    sp500 = obtener_datos_mercado_sp500()

    precios_sem = precios.resample('W').last()
    sp500_sem = sp500.resample('W').last()

    ret_activos_sem = precios_sem.pct_change().dropna()
    ret_sp500_sem = sp500_sem.pct_change().dropna()

    datos_sync = pd.concat([ret_activos_sem, ret_sp500_sem], axis=1, join='inner').dropna()

    if datos_sync.empty:
        ret_activos_final = ret_activos_sem
        ret_sp500_final = pd.Series()
    else:
        ret_activos_final = datos_sync.iloc[:, :-1]
        ret_sp500_final = datos_sync.iloc[:, -1]

    prima_mercado = 0.055
    retornos_finales = pd.Series(index=precios.columns, dtype='float64')
    retornos_capm_teoricos = pd.Series(index=precios.columns, dtype='float64')
    pais_map = {a['ticker']: a['pais'] for a in lista_activos}

    try:
        riesgo_pais_arg = float(prima_riesgo_mas_reciente) / 10000
    except:
        riesgo_pais_arg = 0.0

    for ticker in precios.columns:
        try:
            if not ret_sp500_final.empty and ticker in ret_activos_final.columns:
                cov = np.cov(ret_activos_final[ticker], ret_sp500_final)[0][1]
                var_mercado = np.var(ret_sp500_final, ddof=1)
                beta = cov / var_mercado if var_mercado > 0 else 1.0
            else:
                beta = 1.0

            capm = tasa_libre_riesgo + (beta * prima_mercado)

            if pais_map.get(ticker) == 'Argentina':
                capm += riesgo_pais_arg

            retornos_capm_teoricos[ticker] = capm
        except:
            retornos_capm_teoricos[ticker] = tasa_libre_riesgo

        if ticker in manuales and manuales[ticker] is not None:
            retorno_manual_total = manuales[ticker] / 100
            if retorno_manual_total != 0:
                retorno_anualizado = ((1 + retorno_manual_total) ** (1/5)) - 1
                retornos_finales[ticker] = retorno_anualizado
                continue

        retornos_finales[ticker] = retornos_capm_teoricos[ticker]

    return retornos_finales, retornos_capm_teoricos

def calcular_matriz_covarianza(precios):
    """
    Calcula la matriz de covarianza usando Ledoit-Wolf Shrinkage 
    para reducir error de estimación y mejorar estabilidad.
    """
    retornos_diarios = precios.pct_change().dropna()
    
    lw = LedoitWolf()
    cov_shrunk = lw.fit(retornos_diarios).covariance_
    
    return cov_shrunk * 252

def calcular_cvar_historico(pesos, retornos_diarios, alpha=0.95):
    """
    Calcula el CVaR (Conditional Value at Risk) histórico al nivel alpha.
    Retorna el CVaR anualizado (valor positivo indica pérdida esperada).
    """
    ret_portfolio = retornos_diarios.dot(pesos)
    var_percentil = np.percentile(ret_portfolio, (1 - alpha) * 100)
    cvar = ret_portfolio[ret_portfolio <= var_percentil].mean()
    return -cvar * np.sqrt(252)

def metricas_portfolio(pesos, ret_esp, cov, rf):
    ret = np.dot(pesos, ret_esp)
    vol = np.sqrt(np.dot(pesos.T, np.dot(cov, pesos)))
    sharpe = (ret - rf) / vol if vol > 0 else 0
    return ret, vol, sharpe

def optimizar_portfolios(ret_esp, cov, rf, retornos_diarios):
    """
    Optimiza 3 portfolios:
    1. Máximo Sharpe
    2. Mínima Varianza
    3. Mínimo CVaR (95%)
    """
    n = len(ret_esp)
    bounds = tuple((0, 1) for _ in range(n))
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    init = n * [1./n]

    opt_var = minimize(
        lambda w: np.sqrt(np.dot(w.T, np.dot(cov, w))),
        init, method='SLSQP', bounds=bounds, constraints=constraints
    )

    def neg_sharpe(w):
        r, v, s = metricas_portfolio(w, ret_esp, cov, rf)
        return -s

    opt_sharpe = minimize(
        neg_sharpe, init, method='SLSQP', bounds=bounds, constraints=constraints
    )

    def objetivo_cvar(w):
        return calcular_cvar_historico(w, retornos_diarios, alpha=0.95)

    opt_cvar = minimize(
        objetivo_cvar, init, method='SLSQP', bounds=bounds, constraints=constraints
    )

    return opt_sharpe.x, opt_var.x, opt_cvar.x

def calcular_frontera(ret_esp, cov, ret_min_var, ret_max_asset):
    n = len(ret_esp)
    bounds = tuple((0, 1) for _ in range(n))

    target_returns = np.linspace(ret_min_var, ret_max_asset, 50)
    frontier_vol = []
    frontier_ret = []

    for r in target_returns:
        cons = [
            {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},
            {'type': 'eq', 'fun': lambda x: np.dot(x, ret_esp) - r}
        ]
        res = minimize(
            lambda w: np.sqrt(np.dot(w.T, np.dot(cov, w))),
            n * [1./n], method='SLSQP', bounds=bounds, constraints=cons
        )
        if res.success:
            frontier_vol.append(res.fun)
            frontier_ret.append(r)

    return np.array(frontier_ret), np.array(frontier_vol)

def analizar_portfolio_ui():
    if not activos_portfolio:
        print("❌ No hay activos en el portfolio.")
        return

    style_header = "background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 15px; border-radius: 10px; text-align: center; font-size: 20px; font-weight: bold;"
    display(widgets.HTML(f"<div style='{style_header}'>📊 Asignar Upside Fundamental</div>"))
    display(widgets.HTML("<div style='padding:10px; background:#e3f2fd; color:#1565c0; border-radius:5px; margin:10px 0; text-align:center;'>Ingresa el <b>Upside Total a 5 años (%)</b> para cada activo cargado previamente.<br>Si dejas 0, el modelo usará CAPM.</div>"))

    inputs = {}
    items = []

    for activo in activos_portfolio:
        lbl = widgets.HTML(f"<b>{activo['bandera']} {activo['ticker']}</b>", layout=widgets.Layout(width='150px'))
        val = widgets.FloatText(value=0.0, description='Upside %:', style={'description_width': 'initial'}, layout=widgets.Layout(width='200px'))
        inputs[activo['ticker']] = val
        items.append(widgets.HBox([lbl, val], layout=widgets.Layout(margin='5px', align_items='center')))

    btn_run = widgets.Button(
        description='Ejecutar Optimización',
        layout=widgets.Layout(width='100%', height='50px', margin='20px 0'),
        style={'button_color': '#667eea', 'text_color': 'white', 'font_weight': 'bold'}
    )

    out = widgets.Output()

    def on_click_run(b):
        with out:
            clear_output()
            print("⏳ Ejecutando modelo Quantamental con Ledoit-Wolf y CVaR...")

            for t, widget in inputs.items():
                retornos_esperados_manuales[t] = widget.value

            rf = obtener_tasa_libre_riesgo()
            tickers = [a['ticker'] for a in activos_portfolio]
            precios = descargar_precios_historicos(tickers)

            if precios.empty:
                print("❌ No se pudieron descargar precios.")
                return

            precios = precios.ffill().bfill()
            retornos_diarios = precios.pct_change().dropna()

            ret_esp, ret_capm = calcular_retornos_hibridos(precios, retornos_esperados_manuales, rf, activos_portfolio)
            cov = calcular_matriz_covarianza(precios)

            pesos_sharpe, pesos_minvar, pesos_cvar = optimizar_portfolios(ret_esp, cov, rf, retornos_diarios)
            r_s, v_s, s_s = metricas_portfolio(pesos_sharpe, ret_esp, cov, rf)
            r_m, v_m, s_m = metricas_portfolio(pesos_minvar, ret_esp, cov, rf)
            r_c, v_c, s_c = metricas_portfolio(pesos_cvar, ret_esp, cov, rf)
            
            cvar_s = calcular_cvar_historico(pesos_sharpe, retornos_diarios, 0.95)
            cvar_m = calcular_cvar_historico(pesos_minvar, retornos_diarios, 0.95)
            cvar_c = calcular_cvar_historico(pesos_cvar, retornos_diarios, 0.95)

            f_ret, f_vol = calcular_frontera(ret_esp, cov, r_m, ret_esp.max())

            n_sim = 50000
            w_sim = np.random.random((n_sim, len(tickers)))
            w_sim /= w_sim.sum(axis=1)[:, np.newaxis]
            r_sim = np.dot(w_sim, ret_esp)
            v_sim = np.sqrt(np.einsum('ij,jk,ik->i', w_sim, cov, w_sim))
            s_sim = (r_sim - rf) / v_sim
            
            cvar_sim = np.array([calcular_cvar_historico(w, retornos_diarios, 0.95) for w in w_sim[::100]])

            fig = go.Figure()

            fig.add_trace(go.Scatter(
                x=v_sim*100,
                y=r_sim*100,
                mode='markers',
                marker=dict(
                    color=s_sim,
                    colorscale='Viridis',
                    showscale=True,
                    size=4,
                    colorbar=dict(title="Sharpe Ratio")
                ),
                name='Simulación',
                hovertemplate='Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<extra></extra>'
            ))

            fig.add_trace(go.Scatter(
                x=f_vol*100,
                y=f_ret*100,
                mode='lines',
                line=dict(color='red', width=3),
                name='Frontera Eficiente'
            ))

            max_vol_plot = max(v_sim.max(), v_s * 2) * 100
            y_cml_end = (rf + s_s * (max_vol_plot/100)) * 100
            fig.add_shape(
                type='line',
                x0=0, y0=rf*100,
                x1=max_vol_plot, y1=y_cml_end,
                line=dict(color='orange', width=2, dash='dash')
            )
            fig.add_trace(go.Scatter(
                x=[None], y=[None],
                mode='lines',
                line=dict(color='orange', width=2, dash='dash'),
                name='CML'
            ))

            fig.add_trace(go.Scatter(
                x=[v_s*100],
                y=[r_s*100],
                mode='markers',
                marker=dict(size=20, color='gold', symbol='star', line=dict(color='black', width=2)),
                name='Máximo Sharpe',
                hovertemplate=f'<b>Max Sharpe</b><br>Vol: {v_s*100:.2f}%<br>Ret: {r_s*100:.2f}%<br>Sharpe: {s_s:.2f}<br>CVaR: {cvar_s*100:.2f}%<extra></extra>'
            ))

            fig.add_trace(go.Scatter(
                x=[v_m*100],
                y=[r_m*100],
                mode='markers',
                marker=dict(size=18, color='cyan', symbol='diamond', line=dict(color='black', width=2)),
                name='Mínima Varianza',
                hovertemplate=f'<b>Min Varianza</b><br>Vol: {v_m*100:.2f}%<br>Ret: {r_m*100:.2f}%<br>Sharpe: {s_m:.2f}<br>CVaR: {cvar_m*100:.2f}%<extra></extra>'
            ))

            fig.add_trace(go.Scatter(
                x=[v_c*100],
                y=[r_c*100],
                mode='markers',
                marker=dict(size=18, color='purple', symbol='hexagon', line=dict(color='black', width=2)),
                name='Mínimo CVaR',
                hovertemplate=f'<b>Mínimo CVaR</b><br>Vol: {v_c*100:.2f}%<br>Ret: {r_c*100:.2f}%<br>Sharpe: {s_c:.2f}<br>CVaR: {cvar_c*100:.2f}%<extra></extra>'
            ))

            ind_vol = np.sqrt(np.diag(cov)) * 100
            ind_ret = ret_esp * 100
            fig.add_trace(go.Scatter(
                x=ind_vol,
                y=ind_ret,
                mode='markers+text',
                marker=dict(size=12, color='white', line=dict(color='blue', width=2)),
                text=tickers,
                textposition='top center',
                textfont=dict(size=10, color='black'),
                name='Activos',
                hovertemplate='<b>%{text}</b><br>Vol: %{x:.2f}%<br>Ret: %{y:.2f}%<extra></extra>'
            ))

            fig.update_layout(
                title='<b>Optimización Quantamental con Ledoit-Wolf Shrinkage & CVaR</b>',
                xaxis_title='Riesgo (Volatilidad Anual %)',
                yaxis_title='Retorno Esperado (Anual %)',
                template='plotly_white',
                height=700,
                showlegend=True,
                legend=dict(
                    orientation="v",
                    yanchor="top",
                    y=0.99,
                    xanchor="right",
                    x=0.99,
                    bgcolor="rgba(255, 255, 255, 0.9)",
                    bordercolor="rgba(0, 0, 0, 0.2)",
                    borderwidth=1
                )
            )

            display(HTML(f"""
            <div style="display:flex; gap:15px; justify-content:center; margin-bottom:20px; flex-wrap: wrap;">
                <div style="background:#fff8e1; padding:15px; border-radius:10px; border-left:5px solid #ffd700; width:280px;">
                    <h3>⭐ Máximo Sharpe</h3>
                    <p>Ret: <b>{r_s:.1%}</b> | Vol: <b>{v_s:.1%}</b><br>Sharpe: <b>{s_s:.2f}</b> | CVaR: <b>{cvar_s:.1%}</b></p>
                </div>
                <div style="background:#e0f7fa; padding:15px; border-radius:10px; border-left:5px solid #00bcd4; width:280px;">
                    <h3>💎 Mínima Varianza</h3>
                    <p>Ret: <b>{r_m:.1%}</b> | Vol: <b>{v_m:.1%}</b><br>Sharpe: <b>{s_m:.2f}</b> | CVaR: <b>{cvar_m:.1%}</b></p>
                </div>
                <div style="background:#f3e5f5; padding:15px; border-radius:10px; border-left:5px solid #9c27b0; width:280px;">
                    <h3>🛡️ Mínimo CVaR</h3>
                    <p>Ret: <b>{r_c:.1%}</b> | Vol: <b>{v_c:.1%}</b><br>Sharpe: <b>{s_c:.2f}</b> | CVaR: <b>{cvar_c:.1%}</b></p>
                </div>
            </div>
            """))

            fig.show()

            html_pesos = "<div style='margin-top:30px;'><h3 style='text-align:center; color:#2C3E50; margin-bottom:20px;'>📊 Distribución de Pesos y Retornos</h3>"
            html_pesos += "<table style='width:100%; border-collapse:collapse; box-shadow: 0 2px 4px rgba(0,0,0,0.1);'>"
            html_pesos += "<thead><tr style='background:linear-gradient(135deg, #667eea 0%, #764ba2 100%); color:white;'>"
            html_pesos += "<th style='padding:15px; text-align:left;'>Activo</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Peso Max Sharpe</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Peso Min Var</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Peso Min CVaR</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Tasa CAPM (Teórica)</th>"
            html_pesos += "<th style='padding:15px; text-align:center;'>Retorno Esperado (Final)</th>"
            html_pesos += "</tr></thead><tbody>"

            for i, (t, ps, pm, pc, rcapm, rfinal) in enumerate(zip(tickers, pesos_sharpe, pesos_minvar, pesos_cvar, ret_capm, ret_esp)):
                bg_color = '#f8f9fa' if i % 2 == 0 else '#ffffff'
                html_pesos += f"<tr style='background:{bg_color}; border-bottom:1px solid #dee2e6;'>"
                html_pesos += f"<td style='padding:12px; font-weight:bold; color:#2C3E50;'>{t}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#0066CC;'>{ps:.2%}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#006400;'>{pm:.2%}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#9c27b0;'>{pc:.2%}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#FF6B6B;'>{rcapm:.2%}</td>"
                html_pesos += f"<td style='padding:12px; text-align:center; color:#8B0000; font-weight:bold;'>{rfinal:.2%}</td>"
                html_pesos += "</tr>"

            html_pesos += "</tbody></table></div>"
            display(HTML(html_pesos))

    btn_run.on_click(on_click_run)

    display(widgets.VBox(items))
    display(btn_run)
    display(out)

analizar_portfolio_ui()